In [ ]:
# If your container doesn't have any of the following resources, just run this cell once.
!pip install pymysql
!pip install sqlalchemy
!pip install gensim
import nltk
nltk.download('punkt')

# Exploring the MIMIC Chest X-ray reports

In this notebook, we will take a glance at the MIMIC dataset. Then narrow our scope to the chest x-ray reports. With a few examples of how to interact with MySQL database to pull the data.

In [4]:
# import packages
import pymysql
import pandas as pd
import getpass
from textblob import TextBlob
import re
import scipy
import matplotlib.pyplot as plt
import ipywidgets
from IPython.display import display, HTML
from ipywidgets import interact, interactive, fixed
from sqlalchemy import create_engine
from gensim.parsing.preprocessing import STOPWORDS
from collections import Counter
import sys
%matplotlib inline

### 1. Connect MySQL DB
Set up our MySQL connection with SQL Alchemy (this helps us to read directly into Pandas DataFrames

In [5]:
engine = create_engine('mysql+pymysql://{0}:{1}@{2}:{3}/{4}'
                      .format('jovyan', 
                             getpass.getpass('Enter MySQL password for jovyan'),
                             'mysql',
                             '3306',
                             'mimic2'))

Enter MySQL password for jovyan········


### 2. Explore the MIMIC
Before we move ahead, we will do some counts of patients, admissions and notes to ensure connectivity and also get a sense of the dataset

In [6]:
display(pd.read_sql_query('SELECT count(*) as PatientCount from d_patients', engine))
display(pd.read_sql_query('SELECT count(*) as AdmissionCount from admissions', engine))
display(pd.read_sql_query('SELECT count(*) as NoteCount from noteevents', engine))
display(pd.read_sql_query('SELECT count(DISTINCT code) as Distinct_ICD9_Codes from icd9 LIMIT 5', engine))

,PatientCount
0,4000


,AdmissionCount
0,5074


,NoteCount
0,171927


,Distinct_ICD9_Codes
0,2719


In [7]:
#Let try to pull the most popular ICD9 codes:
display(pd.read_sql_query('SELECT code, description, COUNT(*) counts FROM icd9 GROUP BY code,description ORDER BY counts DESC LIMIT 10 ', engine))

,code,description,counts
0,401.9,UNSPECIFIED ESSENTIAL HYPERTENSION,1441
1,428.0,CONGESTIVE HEART FAILURE UNSPECIFIED,1330
2,427.31,ATRIAL FIBRILLATION,1197
3,518.81,ACUTE RESPIRATORY FAILURE,890
4,584.9,ACUTE RENAL FAILURE UNSPECIFIED,852
5,414.01,CORONARY ATHEROSCLEROSIS OF NATIVE CORONARY AR...,816
6,250.00,DIABETES MELLITUS WITHOUT COMPLICATION TYPE II...,662
7,599.0,URINARY TRACT INFECTION SITE NOT SPECIFIED,606
8,428.0,CONGESTIVE HEART FAILURE \r,515
9,486,PNEUMONIA ORGANISM UNSPECIFIED,487


In [8]:
#Get down to see how to pull notes:
display(pd.read_sql_query('SELECT * from noteevents LIMIT 5', engine))

,subject_id,hadm_id,icustay_id,elemid,charttime,realtime,cgid,correction,cuid,category,title,text,exam_name,patient_info
0,56,28766,None,None,2644-01-17 00:00:00,None,None,None,None,DISCHARGE_SUMMARY,None,\n \n \n \nAdmission Date: [**2644-1-17**] ...,None,None
1,56,28766,None,None,2644-01-17 00:00:00,None,None,None,None,RADIOLOGY_REPORT,None,\n\n\n DATE: [**2644-1-17**] 10:53 AM\n ...,None,None
2,56,28766,None,None,2644-01-17 00:00:00,None,None,None,None,RADIOLOGY_REPORT,None,\n\n\n DATE: [**2644-1-17**] 10:53 AM\n ...,None,None
3,56,28766,None,None,2644-01-17 00:00:00,None,None,None,None,RADIOLOGY_REPORT,None,\n\n\n DATE: [**2644-1-17**] 10:43 AM\n ...,None,None
4,56,28766,None,None,2644-01-17 00:00:00,None,None,None,None,RADIOLOGY_REPORT,None,\n\n\n DATE: [**2644-1-17**] 6:37 AM\n ...,None,None


## 3. Let's find pneumonia patients

In [11]:
# now let's get a dataframe of Patient/Admit/Pneumonia
### This is a useful and efficient query. Thumb up to Kelly.
pneumonia_query = """
    SELECT 
a.subject_id
,a.hadm_id
,a.admit_dt
,(CASE WHEN pneu.HADM_ID IS NOT NULL THEN 1 ELSE 0 END) as Encounter_Pneumonia_Diagnosis
FROM admissions a
LEFT JOIN 
(
    SELECT
    d.HADM_ID
    FROM  icd9 d
    WHERE 
        (code like '486%%')
    GROUP BY d.HADM_ID
) pneu
ON a.HADM_ID = pneu.HADM_ID
"""
pat_admit_pneumonia_df = pd.read_sql_query(pneumonia_query, engine)
display(pat_admit_pneumonia_df.head(10))

,subject_id,hadm_id,admit_dt,Encounter_Pneumonia_Diagnosis
0,56,28766,2644-01-17 00:00:00,0
1,3,2075,2682-09-07 00:00:00,0
2,21,20666,3138-10-29 00:00:00,0
3,21,20882,3139-03-19 00:00:00,0
4,12,12532,2875-09-26 00:00:00,0
5,26,15067,3079-03-03 00:00:00,0
6,37,18052,3264-08-14 00:00:00,1
7,31,15325,2678-08-21 00:00:00,1
8,61,7149,3352-06-23 00:00:00,0
9,61,5712,3353-01-10 00:00:00,0


In [12]:
# let's get a count of how many PNEUMONIA vs NO-PNEUMONIA admits we have
pneumonia_admit_count_df = pat_admit_pneumonia_df.groupby('Encounter_Pneumonia_Diagnosis').size()
display(pneumonia_admit_count_df)

Encounter_Pneumonia_Diagnosis
0    4444
1     630
dtype: int64

## 4. We can then can pull these notes into a frame

In [13]:
# now let's pull a frame of all the FIRST (sorted by text which begins with date) CHEST X-RAY notes
chest_xray_note_query = """
SELECT
subject_id
,hadm_id
,LTRIM(RTRIM(text)) as txt
FROM noteevents
WHERE category = 'RADIOLOGY_REPORT'
    AND (text like '%%CHEST (PORTABLE AP)%%' OR text like '%%CHEST (PA & LAT)%%')
    AND subject_id is not NULL
    AND hadm_id is not NULL        
GROUP BY subject_id, hadm_id, txt
"""
chest_xray_note_df = pd.read_sql_query(chest_xray_note_query, engine)
display(chest_xray_note_df.head(10))

,subject_id,hadm_id,txt
0,3,2075,\n\n\n DATE: [**2682-9-10**] 5:22 AM\n ...
1,3,2075,\n\n\n DATE: [**2682-9-11**] 4:06 PM\n ...
2,3,2075,\n\n\n DATE: [**2682-9-11**] 8:05 AM\n ...
3,3,2075,\n\n\n DATE: [**2682-9-12**] 6:32 AM\n ...
4,3,2075,\n\n\n DATE: [**2682-9-13**] 6:01 AM\n ...
5,3,2075,\n\n\n DATE: [**2682-9-14**] 11:41 PM\n ...
6,3,2075,\n\n\n DATE: [**2682-9-7**] 10:23 PM\n ...
7,3,2075,\n\n\n DATE: [**2682-9-7**] 6:16 PM\n ...
8,3,2075,\n\n\n DATE: [**2682-9-8**] 1:00 AM\n ...
9,3,2075,\n\n\n DATE: [**2682-9-8**] 4:43 PM\n ...


Much like a SQL "join" we can combine our frame which has ICD-9 codes with the frame that has notes so that we can sample from these intelligently

In [137]:
pneumonia_note_df = pd.merge(pat_admit_pneumonia_df, chest_xray_note_df, on = ['subject_id', 'hadm_id'])
pneumonia_note_df[:5]

,subject_id,hadm_id,admit_dt,Encounter_Pneumonia_Diagnosis,txt
0,56,28766,2644-01-17 00:00:00,0,\n\n\n DATE: [**2644-1-17**] 6:37 AM\n ...
1,56,28766,2644-01-17 00:00:00,0,\n\n\n DATE: [**2644-1-19**] 12:09 PM\n ...
2,3,2075,2682-09-07 00:00:00,0,\n\n\n DATE: [**2682-9-10**] 5:22 AM\n ...
3,3,2075,2682-09-07 00:00:00,0,\n\n\n DATE: [**2682-9-11**] 4:06 PM\n ...
4,3,2075,2682-09-07 00:00:00,0,\n\n\n DATE: [**2682-9-11**] 8:05 AM\n ...


In [138]:
pneumonia_note_df[-5:]

,subject_id,hadm_id,admit_dt,Encounter_Pneumonia_Diagnosis,txt
19421,32763,33007,2724-06-05 00:00:00,1,\n\n\n DATE: [**2724-6-30**] 11:59 PM\n ...
19422,32763,33007,2724-06-05 00:00:00,1,\n\n\n DATE: [**2724-6-5**] 7:04 PM\n ...
19423,32763,33007,2724-06-05 00:00:00,1,\n\n\n DATE: [**2724-6-8**] 8:02 AM\n ...
19424,32763,33007,2724-06-05 00:00:00,1,\n\n\n DATE: [**2724-6-9**] 2:56 AM\n ...
19425,32763,33007,2724-06-05 00:00:00,1,\n\n\n DATE: [**2724-7-8**] 3:38 AM\n ...


## 5. Some ideas on selecting which reports to use 
![image.png](./Slide_pngs/Slide16.png)

## 6. What sample size should you use?
![image.png](./Slide_pngs/Slide18.png)

## 7. Example Code For Stratified Sampling


We sampled notes where the encounter was coded for Pneumonia (ICD-9 code 486.* ) and where it was not coded.  We performed stratified sampling of a percentage of notes from one and the remainder from the other.

In [119]:
# select notes from the encounters without code
group0_notes=pneumonia_note_df.loc[pneumonia_note_df['Encounter_Pneumonia_Diagnosis'] == 0]
# select notes from the encounters with code
group1_notes=pneumonia_note_df.loc[pneumonia_note_df['Encounter_Pneumonia_Diagnosis'] == 1]
# sampling within each group and then shuffle
samples=pd.concat([group0_notes.sample(10),group1_notes.sample(30)]).sample(frac=1)
samples.shape

(40, 5)

In [120]:
# display the first 10 notes
samples.head(10)

,subject_id,hadm_id,admit_dt,Encounter_Pneumonia_Diagnosis,txt
9723,14237,10911,3134-11-12 00:00:00,0,\n\n\n DATE: [**3135-1-31**] 9:28 AM\n ...
4065,5695,12073,2804-03-19 00:00:00,1,\n\n\n DATE: [**2804-3-19**] 10:05 PM\n ...
5530,8011,18602,2932-09-11 00:00:00,1,\n\n\n DATE: [**2932-10-4**] 2:33 PM\n ...
16725,24431,21755,2726-09-19 00:00:00,1,\n\n\n DATE: [**2726-9-27**] 7:25 AM\n ...
14319,20553,10482,3320-12-23 00:00:00,1,\n\n\n DATE: [**3320-12-31**] 3:06 PM\n ...
6862,10076,6077,2850-02-14 00:00:00,1,\n\n\n DATE: [**2850-2-21**] 7:05 AM\n ...
16128,23395,2005,3208-05-14 00:00:00,1,\n\n\n DATE: [**3208-5-18**] 9:45 AM\n ...
6668,9773,27156,3157-03-13 00:00:00,0,\n\n\n DATE: [**3157-3-15**] 11:35 PM\n ...
16294,23641,677,2610-06-20 00:00:00,0,\n\n\n DATE: [**2610-7-4**] 10:31 AM\n ...
19335,32618,30229,3429-08-16 00:00:00,1,\n\n\n DATE: [**3429-8-17**] 9:47 AM\n ...


<br/><br/>This material presented as part of the DeCART Data Science for the Health Science Summer Program at the University of Utah in 2018.<br/>
Presenters : Dr. Wendy Chapman, Jianlin Shi <br> Acknowledgement: Many thanks to Kelly Peterson and Olga Patterson because part of the materials are adopted from his previous work.